In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15

In [70]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [71]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://zkt2018-challenge.s3.us-east-2.amazonaws.com/vine_table.csv"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("vine_table.csv"), inferSchema=True, sep=',')

# Show DataFrame
df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1NQOI19PNB1KJ|          3|            0|          0|   N|                Y|
|R1BEN5Q26EIK98|          1|            2|          4|   N|                Y|
|R3K6QS6S5X3GEA|          3|            0|          1|   N|                Y|
|R24M3PR92MMQN0|          1|            2|          2|   N|                N|
|R3M0UCU8IYLJIK|          5|            0|          3|   N|                Y|
|R30LT16S75T7IU|          2|            2|          5|   N|                Y|
|R2YRN4ZU4TLJ85|          3|            3|          3|   N|                Y|
|R2GFO14E9LXRUN|          4|            0|          0|   N|                Y|
|R105U1B8VAALBE|          1|            4|          4|   N|                Y|
|R360RFQU82ST0M|          1|           14|         20|   N|     

### Create DataFrames to match tables

In [72]:
from pyspark.sql.functions import to_date

In [73]:
# Create the vine_table DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1NQOI19PNB1KJ|          3|            0|          0|   N|                Y|
|R1BEN5Q26EIK98|          1|            2|          4|   N|                Y|
|R3K6QS6S5X3GEA|          3|            0|          1|   N|                Y|
|R24M3PR92MMQN0|          1|            2|          2|   N|                N|
|R3M0UCU8IYLJIK|          5|            0|          3|   N|                Y|
|R30LT16S75T7IU|          2|            2|          5|   N|                Y|
|R2YRN4ZU4TLJ85|          3|            3|          3|   N|                Y|
|R2GFO14E9LXRUN|          4|            0|          0|   N|                Y|
|R105U1B8VAALBE|          1|            4|          4|   N|                Y|
|R360RFQU82ST0M|          1|           14|         20|   N|     

In [74]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
filtered_votes_df = vine_df.filter(col('total_votes') >= 20)
filtered_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R360RFQU82ST0M|          1|           14|         20|   N|                N|
|R37239AU42EJVJ|          3|           20|         20|   N|                N|
|R3W1AV85URGDJV|          1|           12|         20|   N|                Y|
|R1F98PQXG08UMW|          1|           13|         23|   N|                Y|
|R3R5BGKR70CYOP|          1|           51|         51|   N|                Y|
| RFQQZOKX9JYMZ|          5|           23|         25|   N|                N|
|R2U8HMTZAIC6O7|          1|           16|         22|   N|                Y|
|R1NSWH1JFISETA|          1|           16|         20|   N|                Y|
|R2WBO1I3UM4DWW|          1|           48|         53|   N|                N|
|R3T79742JOKTGH|          1|            3|         22|   N|     

In [75]:
# Filter the dataframe to get helpful votes
hf_votes_df = filtered_votes_df.filter((filtered_votes_df.helpful_votes/filtered_votes_df.total_votes)>= 0.5)
hf_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R360RFQU82ST0M|          1|           14|         20|   N|                N|
|R37239AU42EJVJ|          3|           20|         20|   N|                N|
|R3W1AV85URGDJV|          1|           12|         20|   N|                Y|
|R1F98PQXG08UMW|          1|           13|         23|   N|                Y|
|R3R5BGKR70CYOP|          1|           51|         51|   N|                Y|
| RFQQZOKX9JYMZ|          5|           23|         25|   N|                N|
|R2U8HMTZAIC6O7|          1|           16|         22|   N|                Y|
|R1NSWH1JFISETA|          1|           16|         20|   N|                Y|
|R2WBO1I3UM4DWW|          1|           48|         53|   N|                N|
|R32PF84PMNRVJI|          5|           91|         98|   N|     

In [76]:
# Filter the dataframe to get the Vine program reviews
vine_votes_df = hf_votes_df.filter(col('vine')=='Y')
vine_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3SL0Q3AVX4J3Y|          5|           34|         35|   Y|                N|
|R1ANZ8I6C0DT0A|          4|           24|         25|   Y|                N|
|R39794RGHXLMV8|          5|           40|         46|   Y|                N|
|R2LVP49XP5MPED|          4|           34|         36|   Y|                N|
| RQTPBMNOJGVNO|          3|           27|         29|   Y|                N|
|R1KN5DA53079V5|          5|           32|         35|   Y|                N|
|R3DL2LMDGKYDPT|          4|           22|         25|   Y|                N|
| R22GXQ8OXTXTW|          5|           50|         53|   Y|                N|
|R14M63V32IEHDG|          4|           61|         63|   Y|                N|
| RF0DXDE5UHH55|          5|          209|        223|   Y|     

In [77]:
# Filter the dataframe to get the non-Vine program reviews
nonvine_votes_df = hf_votes_df.filter(col('vine')=='N')
nonvine_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R360RFQU82ST0M|          1|           14|         20|   N|                N|
|R37239AU42EJVJ|          3|           20|         20|   N|                N|
|R3W1AV85URGDJV|          1|           12|         20|   N|                Y|
|R1F98PQXG08UMW|          1|           13|         23|   N|                Y|
|R3R5BGKR70CYOP|          1|           51|         51|   N|                Y|
| RFQQZOKX9JYMZ|          5|           23|         25|   N|                N|
|R2U8HMTZAIC6O7|          1|           16|         22|   N|                Y|
|R1NSWH1JFISETA|          1|           16|         20|   N|                Y|
|R2WBO1I3UM4DWW|          1|           48|         53|   N|                N|
|R32PF84PMNRVJI|          5|           91|         98|   N|     

In [78]:
from pyspark.sql.functions import count
# Determine total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review
all_reviews = hf_votes_df.select(col('review_id')).count()
all_reviews

17762

In [79]:
five_star_count = hf_votes_df.filter(col('star_rating')==5).count()
five_star_count


5256

In [80]:
five_star_vine = vine_votes_df.select(col('star_rating')==5).count()
percentage_reviews_vine = (five_star_vine/count_reviews)*100
percentage_reviews_vine


1.3962391622565027

In [81]:
five_star_nonvine = nonvine_votes_df.select(col('star_rating')==5).count()
percentage_reviews_nonvine = (five_star_nonvine/count_reviews)*100
percentage_reviews_nonvine

98.6037608377435